## Contexte du notebook
Ce notebook fait partie d'un projet d'analyse des accidents autour des feux de circulation. 
Il documente la preparation des donnees, le rattachement geographique 
et la production de tables pour Power BI.


# Rapport d'analyse des accidents autour des feux de circulation

## Introduction
Cette analyse vise a etudier la distribution des accidents de la route a proximite des feux de circulation a Montreal. 
L'objectif est de comprendre ou et quand les accidents sont les plus frequents pour guider des actions de prevention 
et d'amelioration de la securite routiere.

Les donnees proviennent des jeux de donnees municipaux suivants :
- Collisions routieres (accidents)
- Feux de circulation

Sources : https://donnees.montreal.ca (jeux de donnees "Collisions routieres" et "Feux de circulation").

Ce rapport decrit les variables, effectue le nettoyage, rattache les accidents aux feux les plus proches 
et presente des indicateurs descriptifs. Une etape suivante consistera a produire des visualisations 
et des analyses plus poussees (ex. tendances temporelles, facteurs de risque, comparaison par secteur).


## Sommaire
- Introduction
- Description des variables
- Chargement des donnees
- Exploration ciblee (EDA legere)
- Methodologie de rattachement
- Jointure geographique
- Creation des tables analytiques
- Indicateurs descriptifs
- Limites et biais
- Export pour Power BI
- Suite / Power BI


## Description des variables
Les variables principales utilisees dans l'analyse sont :

**Accidents (collisions)**
- `DT_ACCDN` : date de l'accident
- `HEURE_ACCDN` : plage horaire de l'accident (ex. 15:00:00-15:59:00)
- `LOC_LAT`, `LOC_LONG` : coordonnees geographiques de l'accident

Variables derivees :
- `heure` : heure extraite de `HEURE_ACCDN`
- `annee`, `mois`, `jour`, `jour_semaine` : composantes temporelles derivees

**Feux de circulation**
- `INT_NO` / `ID_FEU` : identifiant du feu (ID_FEU = INT_NO)
- `Latitude`, `Longitude` : coordonnees geographiques
- `ARRONDISSEMENT`, `RUE_1`, `RUE_2` : localisation et contexte

**Variables de rattachement**
- `dist_m` : distance entre un accident et le feu le plus proche
- `ID_FEU` : feu associe a chaque accident apres rattachement


## 1. Chargement des donnees
Chargement des fichiers sources sans transformation lourde.


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA_DIR = Path("..") / "data" / "raw"
feux_path = DATA_DIR / "feux-circulation.csv"
acc_path = DATA_DIR / "collisions_routieres.csv"

feux = pd.read_csv(feux_path)
acc = pd.read_csv(acc_path)

print('Feux:', feux.shape)
print('Accidents:', acc.shape)
display(feux.head(3))
display(acc.head(3))


Feux: (2345, 10)
Accidents: (218272, 68)


,INT_NO,RUE_1,RUE_2,NO_ARRONDISSEMENT,ARRONDISSEMENT,PERMANENT_OU_TEMPORAIRE,LOC_X,LOC_Y,Longitude,Latitude
0,146,Côte-des-Neiges,La Peltrie,7,Côte-des-Neiges - Notre-Dame-de-Grâce,Permanent,294637.8440,5040004.896,-73.630033,45.499892
1,147,Côte-des-Neiges,Linton,7,Côte-des-Neiges - Notre-Dame-de-Grâce,Permanent,294564.8675,5040052.122,-73.630968,45.500316
2,148,Appleton,Côte-des-Neiges/Plamondon,7,Côte-des-Neiges - Notre-Dame-de-Grâce,Permanent,294256.2500,5040250.000,-73.634921,45.502092


,NO_SEQ_COLL,JR_SEMN_ACCDN,DT_ACCDN,CD_MUNCP,NO_CIVIQ_ACCDN,SFX_NO_CIVIQ_ACCDN,BORNE_KM_ACCDN,RUE_ACCDN,TP_REPRR_ACCDN,ACCDN_PRES_DE,...,NB_VICTIMES_VELO,VITESSE_AUTOR,LOC_X,LOC_Y,LOC_COTE_QD,LOC_COTE_PD,LOC_DETACHEE,LOC_IMPRECISION,LOC_LONG,LOC_LAT
0,SPVM _ 2012 _ 1,ME,2012-02-01,66102.0,3501.0,NaN,NaN,ST CHARLES,2.0,STAT,...,0,NaN,276517.3795,5035127.484,A,3,O,N,-73.861616,45.455505
1,SPVM _ 2012 _ 10,MA,2012-01-03,66023.0,NaN,NaN,NaN,TERR VILLE DE MTL,NaN,NaN,...,0,NaN,275209.4619,5038619.375,B,4,N,N,-73.878549,45.486871
2,SPVM _ 2012 _ 100,VE,2012-02-24,66023.0,NaN,NaN,NaN,JACQUES BIZARD,1.0,CHERRIER,...,0,50.0,275759.0790,5039027.153,A,1,N,O,-73.871542,45.490564


### Description rapide des datasets
- Feux: chaque ligne represente un feu de circulation a une intersection de Montreal; base spatiale du projet.
- Accidents: chaque ligne represente une collision (date, heure, coordonnees); base evenementielle.


## 2. Exploration ciblee (EDA legere)
Verifier uniquement ce qui est utile pour la liaison feux-accidents.


### Traitement de la variable heure
La colonne `HEURE_ACCDN` contient des plages horaires. On extrait l'heure de debut et on exclut les valeurs non precisees.


In [2]:
# Accidents
import unicodedata

acc = acc.copy()
acc["DT_ACCDN"] = pd.to_datetime(acc["DT_ACCDN"], errors="coerce")

# HEURE_ACCDN est un libelle de plage horaire (ex: 15:00:00-15:59:00)
heure_raw = acc["HEURE_ACCDN"].astype(str)
heure_norm = (
    heure_raw.apply(lambda s: unicodedata.normalize("NFKD", s))
    .str.encode("ascii", errors="ignore")
    .str.decode("ascii")
)
valid_mask = ~heure_norm.str.contains("Non prec", case=False, na=False)
heure_start = (
    heure_raw.where(valid_mask)
    .str.split("-", n=1, expand=True)[0]
)
acc["heure"] = pd.to_datetime(heure_start, errors="coerce").dt.hour

acc["annee"] = acc["DT_ACCDN"].dt.year
acc["mois"] = acc["DT_ACCDN"].dt.month
acc["jour"] = acc["DT_ACCDN"].dt.day
acc["jour_semaine"] = acc["DT_ACCDN"].dt.day_name()

acc = acc.dropna(subset=["LOC_LAT", "LOC_LONG"])

# Feux
feux = feux.copy()
feux = feux.dropna(subset=["Latitude", "Longitude"])
feux["ID_FEU"] = feux["INT_NO"]

print("Accidents apres nettoyage:", acc.shape)
print("Feux apres nettoyage:", feux.shape)


Accidents apres nettoyage: (218261, 73)
Feux apres nettoyage: (2345, 11)


/var/folders/sd/6lbm6rjd3z586_j02tfr3fzr0000gn/T/ipykernel_27192/2846085158.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc["heure"] = pd.to_datetime(heure_start, errors="coerce").dt.hour


In [3]:
# Accidents
import unicodedata

acc = acc.copy()
acc["DT_ACCDN"] = pd.to_datetime(acc["DT_ACCDN"], errors="coerce")

# HEURE_ACCDN est un libelle de plage horaire (ex: 15:00:00-15:59:00)
heure_raw = acc["HEURE_ACCDN"].astype(str)
heure_norm = (
    heure_raw.apply(lambda s: unicodedata.normalize('NFKD', s))
    .str.encode('ascii', errors='ignore')
    .str.decode('ascii')
)
valid_mask = ~heure_norm.str.contains('Non prec', case=False, na=False)
heure_start = heure_raw.where(valid_mask).str.split('-', n=1, expand=True)[0]
acc["heure"] = pd.to_datetime(heure_start, errors="coerce").dt.hour

acc["annee"] = acc["DT_ACCDN"].dt.year
acc["mois"] = acc["DT_ACCDN"].dt.month
acc["jour"] = acc["DT_ACCDN"].dt.day
acc["jour_semaine"] = acc["DT_ACCDN"].dt.day_name()

acc = acc.dropna(subset=["LOC_LAT", "LOC_LONG"])

# Feux
feux = feux.copy()
feux = feux.dropna(subset=["Latitude", "Longitude"])
feux["ID_FEU"] = feux["INT_NO"]

print('Accidents apres nettoyage:', acc.shape)
print('Feux apres nettoyage:', feux.shape)


Accidents apres nettoyage: (218261, 73)
Feux apres nettoyage: (2345, 11)


/var/folders/sd/6lbm6rjd3z586_j02tfr3fzr0000gn/T/ipykernel_27192/592817890.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc["heure"] = pd.to_datetime(heure_start, errors="coerce").dt.hour


In [4]:
# Accidents
import unicodedata

acc = acc.copy()
acc["DT_ACCDN"] = pd.to_datetime(acc["DT_ACCDN"], errors="coerce")

# HEURE_ACCDN est souvent une plage horaire (ex: 15:00:00-15:59:00)
heure_raw = acc["HEURE_ACCDN"]
heure_str = heure_raw.astype(str)
heure_norm = (
    heure_str.apply(lambda s: unicodedata.normalize("NFKD", s))
    .str.encode("ascii", errors="ignore")
    .str.decode("ascii")
)
valid_mask = ~heure_norm.str.contains("Non prec", case=False, na=False)
heure_start = heure_str.where(valid_mask).str.split("-", n=1, expand=True)[0]
heure_parsed = pd.to_datetime(heure_start, errors="coerce").dt.hour
heure_num = pd.to_numeric(heure_raw, errors="coerce")
acc["heure"] = heure_parsed.fillna(heure_num).astype("Int64")

acc["annee"] = acc["DT_ACCDN"].dt.year
acc["mois"] = acc["DT_ACCDN"].dt.month
acc["jour"] = acc["DT_ACCDN"].dt.day
acc["jour_semaine"] = acc["DT_ACCDN"].dt.day_name()

acc = acc.dropna(subset=["LOC_LAT", "LOC_LONG"])

# Feux
feux = feux.copy()
feux = feux.dropna(subset=["Latitude", "Longitude"])
feux["ID_FEU"] = feux["INT_NO"]

print('Accidents apres nettoyage:', acc.shape)
print('Feux apres nettoyage:', feux.shape)


/var/folders/sd/6lbm6rjd3z586_j02tfr3fzr0000gn/T/ipykernel_27192/359420392.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  heure_parsed = pd.to_datetime(heure_start, errors="coerce").dt.hour


Accidents apres nettoyage: (218261, 73)
Feux apres nettoyage: (2345, 11)


## 3. Methodologie de rattachement
Chaque accident est associe au feu le plus proche via une recherche de voisin le plus proche 
(BallTree) sur les coordonnees en radians. 
Un rayon maximal est fixe a partir du 98e percentile des distances et plafonne a 200 m 
pour eviter les associations trop lointaines.


### Hypothese methodologique
Les accidents sont rattaches au feu de circulation le plus proche pour analyser la concentration spatiale autour des intersections controlees.


## 4. Jointure geographique (coeur du projet)
Rattacher chaque accident au feu de circulation le plus proche.


In [5]:
from sklearn.neighbors import BallTree

# Coordonnees en radians
feux_rad = np.deg2rad(feux[["Latitude", "Longitude"]].values)
acc_rad = np.deg2rad(acc[["LOC_LAT", "LOC_LONG"]].values)

tree = BallTree(feux_rad, metric='haversine')
dist_rad, idx = tree.query(acc_rad, k=1)
dist_m = dist_rad[:, 0] * 6371000

# Rayon adaptatif: on garde la grande majorite des accidents
quantile_target = 0.98
radius_m = float(np.quantile(dist_m, quantile_target))
radius_m = min(radius_m, 200.0)

acc["dist_m"] = dist_m
acc["ID_FEU"] = feux.iloc[idx[:, 0]]["ID_FEU"].values

acc_geo = acc[acc["dist_m"] <= radius_m].copy()

print(f'Rayon retenu: {radius_m:.1f} m')
print('Accidents rattaches:', acc_geo.shape[0])
print('Taux de rattachement:', acc_geo.shape[0] / acc.shape[0])


Rayon retenu: 200.0 m
Accidents rattaches: 163797
Taux de rattachement: 0.7504638941450832


## 5. Creation des tables analytiques
Tables concises destinees au tableau de bord.


In [6]:
# Jointure des infos feux pour contexte (arrondissement, rues)
acc_geo = acc_geo.merge(
    feux[["ID_FEU", "ARRONDISSEMENT", "RUE_1", "RUE_2", "Latitude", "Longitude"]],
    on="ID_FEU",
    how="left"
)

accidents_par_feu = (
    acc_geo.groupby(["ID_FEU", "ARRONDISSEMENT", "RUE_1", "RUE_2"])
    .size()
    .reset_index(name="nb_accidents")
)

accidents_par_feu_heure = (
    acc_geo.groupby(["ID_FEU", "heure"])
    .size()
    .reset_index(name="nb_accidents")
)

accidents_par_jour = (
    acc_geo.groupby(["DT_ACCDN"])
    .size()
    .reset_index(name="nb_accidents")
)

display(accidents_par_feu.head(5))


,ID_FEU,ARRONDISSEMENT,RUE_1,RUE_2,nb_accidents
0,1,Sud-Ouest,Charlevoix,Notre-Dame,129
1,2,Ville-Marie,Docteur-Penfield,Peel,86
2,3,Côte-des-Neiges - Notre-Dame-de-Grâce,Borden,Somerled,35
3,4,Côte-des-Neiges - Notre-Dame-de-Grâce,Cavendish,Côte-Saint-Luc,112
4,5,Côte-des-Neiges - Notre-Dame-de-Grâce,Cavendish,Fielding,57


## 6. Indicateurs descriptifs (rapides)
Quelques indicateurs simples pour appuyer le storytelling, sans interpretation causale.


In [7]:
top_feux = accidents_par_feu.sort_values('nb_accidents', ascending=False).head(10)
display(top_feux)

accidents_par_heure = (
    acc_geo.groupby(["heure"])
    .size()
    .reset_index(name="nb_accidents")
    .sort_values('heure')
)
display(accidents_par_heure.head(10))


,ID_FEU,ARRONDISSEMENT,RUE_1,RUE_2,nb_accidents
1267,1306,Villeray - Saint-Michel - Parc-Extension,Crémazie,Saint-Michel inter. Sud-Est,671
908,938,Mercier - Hochelaga - Maisonneuve,Sherbrooke,Trianon,564
1637,1730,Anjou,Galeries d'Anjou,Sortie Centre-d'achats / Halles,480
100,104,Côte-des-Neiges - Notre-Dame-de-Grâce,Décarie,Paré inter. Est,417
2147,15079,LaSalle,Guy-Bouchard,Newman,395
1227,1264,Ahuntsic - Cartierville,Acadie,Crémazie bretelle Nord-Est,395
96,100,Côte-des-Neiges - Notre-Dame-de-Grâce,Décarie,Jean-Talon inter. Est,367
2090,13268,Saint-Laurent,Côte-de-Liesse,Sainte-Croix,359
1820,6735,Saint-Léonard,Langelier,Métropolitain Nord,338
1934,8884,Montréal-Nord,Henri-Bourassa,Rolland,338


,heure,nb_accidents
0,0,3210
1,1,2222
2,2,2081
3,3,2854
4,4,1633
5,5,1604
6,6,3069
7,7,6067
8,8,8480
9,9,7686


## 7. Limites et biais
- Les accidents sans coordonnees sont exclus.
- Les heures "Non precise" sont retirees, ce qui peut sous-estimer certains periodes.
- Le choix du rayon peut influencer le rattachement dans les zones denses.
- L'association par proximite ne prouve pas un lien causal avec le feu.


## 8. Export pour Power BI
Export des tables analytiques en CSV.


## 9. Suite / Power BI
Les fichiers exportes servent a construire des tableaux de bord : 
top feux par accidents, distribution par heure/jour, et analyses par arrondissement. 
La suite prevue inclut des visualisations cartographiques et des comparaisons temporelles.


In [8]:
OUT_DIR = Path("..") / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

acc_geo.to_csv(OUT_DIR / "accidents_avec_feux.csv", index=False)
accidents_par_feu.to_csv(OUT_DIR / "accidents_par_feu.csv", index=False)
accidents_par_feu_heure.to_csv(OUT_DIR / "accidents_par_feu_heure.csv", index=False)
accidents_par_jour.to_csv(OUT_DIR / "accidents_par_jour.csv", index=False)

print('Exports termines:', list(OUT_DIR.glob('*.csv')))


Exports termines: [PosixPath('../data/processed/accidents_par_feu.csv'), PosixPath('../data/processed/accidents_par_jour.csv'), PosixPath('../data/processed/accidents_avec_feux.csv'), PosixPath('../data/processed/accidents_par_feu_heure.csv')]
